# DSS Thesis - Koen de Bonth

### Import packages

In [9]:
import os
import sys
from pathlib import Path

# Get the current working directory
current_dir = os.getcwd()

# Set the root directory to the parent of the current directory
root_dir = Path(current_dir).parent

# Add the root directory to sys.path so Python can find the utils module
sys.path.append(str(root_dir))
print(f"Added {root_dir} to Python path")


Added c:\Thesis\DSS_Thesis_CNC\DSS_Thesis_CNC to Python path


In [10]:
from utils import data_loader_utils
import itertools 
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
import numpy as np
from scipy import signal,stats
from tqdm import tqdm

### Loading and Preparing Data

In [11]:
machines = ["M01","M02","M03"]
process_names = ["OP00","OP01","OP02","OP03","OP04","OP05","OP06","OP07","OP08","OP09","OP10","OP11","OP12","OP13","OP14"]
labels = ["good","bad"]

path_to_dataset = os.path.join(root_dir, "data")

X_data = []
y_data = []

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    data_list, data_label = data_loader_utils.load_tool_research_data(data_path, label=label)
    X_data.extend(data_list)
    y_data.extend(data_label)


### Metadata Extraction

In [12]:
# List to store metadata about each h5 file
file_metadata = []

# Sampling rate in Hz for the measurements
SAMPLING_RATE = 2000  

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    if os.path.exists(data_path):
        for file in os.listdir(data_path):
            if file.endswith('.h5'):
                fullpath = os.path.join(data_path, file)
                with h5py.File(fullpath, 'r') as f:
                    dataset_name = list(f.keys())[0]
                    dataset = f[dataset_name]
                    
                    file_metadata.append({
                        'machine': machine,
                        'operation': process_name,
                        'class': label,
                        'measurements': dataset.shape[0],
                        'channels': dataset.shape[1] if len(dataset.shape) > 1 else 1,
                        'duration_sec': dataset.shape[0] / SAMPLING_RATE,  # Duration in seconds
                        'duration_min': dataset.shape[0] / SAMPLING_RATE / 60,  # Duration in minutes 
                        'file_size_mb': os.path.getsize(fullpath) / (1024 * 1024),
                        'month_created': file.split('_')[1],
                        'year_created': file.split('_')[2],
                        'full_path': fullpath
                    })

# Create dataframe with file metadata
df_measurement_files = pd.DataFrame(file_metadata)
df_measurement_files.to_csv('export/measurement_files_metadata.csv', index=False)

# display(df_measurement_files)

### Feature Extraction

In [13]:
def extract_wavelet_features(signal, wavelet='coif8', max_level=3):
    """
    Perform wavelet packet decomposition and extract statistical features
    
    Parameters:
    -----------
    signal : ndarray
        Input signal (1D array)
    wavelet : str
        Wavelet to use (default: 'coif8')
    max_level : int
        Maximum decomposition level
        
    Returns:
    --------
    dict: Dictionary of statistical features
    """
    # Create wavelet packet
    wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=max_level)
    
    # Extract nodes at the maximum level
    level_nodes = [node.path for node in wp.get_level(max_level, 'natural')]
    
    features = {}
    
    for node in level_nodes:
        # Get coefficients for this node
        coeffs = wp[node].data
        
        # Extract statistical features
        features[f"mean_{node}"] = np.mean(coeffs)
        features[f"max_{node}"] = np.max(coeffs)
        features[f"min_{node}"] = np.min(coeffs)
        features[f"std_{node}"] = np.std(coeffs)
        features[f"kurtosis_{node}"] = stats.kurtosis(coeffs)
        features[f"skewness_{node}"] = stats.skew(coeffs)
        
        # Shannon entropy
        # Normalize the coefficients
        coeffs_norm = np.abs(coeffs) / np.sum(np.abs(coeffs) + 1e-10)
        entropy = -np.sum(coeffs_norm * np.log2(coeffs_norm + 1e-10))
        features[f"entropy_{node}"] = entropy
        
    return features

In [14]:
# List to store all features
all_features = []

# Process each sample in X_data
for i, sample in enumerate(tqdm(X_data, desc="Extracting features")):
    sample_features = {}
    
    # Process each axis (channel)
    for axis in range(sample.shape[1]):
        # Get signal for this axis
        signal = sample[:, axis]
        
        # Apply wavelet packet transform and extract features
        wp_features = extract_wavelet_features(signal, wavelet='coif8', max_level=3)
        
        # Add axis identifier to feature names
        for key, value in wp_features.items():
            sample_features[f"axis{axis}_{key}"] = value
    
    # Add label
    split_label = y_data[i].split("_")
    sample_features['label'] = split_label[-1]
    sample_features['machine'] = split_label[0]
    
    # Add to collection
    all_features.append(sample_features)

# Convert to DataFrame
features_df = pd.DataFrame(all_features)

Extracting features:  57%|█████▋    | 970/1702 [00:30<00:17, 41.23it/s]

In [15]:
features_df

,axis0_mean_aaa,axis0_max_aaa,axis0_min_aaa,axis0_std_aaa,axis0_kurtosis_aaa,axis0_skewness_aaa,axis0_entropy_aaa,axis0_mean_aad,axis0_max_aad,axis0_min_aad,...,axis2_skewness_dda,axis2_entropy_dda,axis2_mean_ddd,axis2_max_ddd,axis2_min_ddd,axis2_std_ddd,axis2_kurtosis_ddd,axis2_skewness_ddd,axis2_entropy_ddd,label
0,-10.991404,4541.231502,-4349.812566,1310.989218,3.684345,0.002241,13.161231,0.152372,499.545762,-439.560444,...,0.013782,14.686282,-0.223901,662.641629,-926.278896,38.070083,37.578280,-0.909573,14.167624,good
1,-11.567185,4387.715013,-4330.328997,1311.196879,3.689392,0.000579,13.158127,0.118528,477.866345,-487.383820,...,-0.000660,14.676513,0.197712,880.045329,-728.211535,37.401571,33.879701,0.624161,14.182913,good
2,-11.391728,4653.423750,-4785.838722,1311.332192,3.695720,-0.008125,13.168987,-0.017426,505.525783,-619.114509,...,0.096591,14.648069,-0.018739,634.223970,-480.199034,36.460751,24.238390,0.080482,14.126589,good
3,-13.518927,4415.331945,-4402.626471,1316.006555,3.616536,0.007331,13.155008,0.060135,490.759898,-534.575650,...,-0.119462,14.353383,0.089371,618.587804,-710.922108,37.095871,30.898552,-0.218302,14.104426,good
4,-14.244809,4391.101813,-4322.837682,1310.944257,3.673275,0.008677,13.176719,0.075658,490.240612,-467.977956,...,-0.432880,14.261143,0.118642,462.380058,-779.346596,36.324785,26.889594,-0.185641,14.130760,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,-4.702717,4071.524328,-4177.332966,1024.363072,6.216405,-0.089596,10.870899,-0.041020,515.926848,-643.379333,...,0.130507,12.119109,-0.156913,1221.222739,-1357.675443,73.352013,59.203714,-0.091659,12.401100,good
1698,-2.637277,4016.117938,-4209.745565,1033.189206,6.082270,-0.092794,10.910870,0.178519,466.681280,-557.107835,...,0.036515,12.043336,0.668770,1217.127790,-1273.067288,67.001386,65.040887,0.602595,12.348593,good
1699,-3.367784,4063.473369,-4133.360324,1032.542725,6.087124,-0.091802,10.863060,-0.340412,572.242368,-545.926165,...,0.026369,11.972986,0.549298,1157.694011,-1272.376346,59.196145,80.164848,0.979685,12.346798,good
1700,57.288906,3982.278809,-3820.776611,1235.220337,3.234195,-0.018610,11.087244,2.149157,1219.704834,-1136.094238,...,0.202069,11.920391,-2.007525,940.493103,-1150.989502,128.918106,10.416579,-0.491225,11.645780,bad


### Exploratory Data Analysis (EDA)

In [16]:
# # Plot coif8 en db14 over elkaar voor vergelijking
# plt.figure(figsize=(15, 5))

# signal = x[100][:2000, 0]  # Gebruik zelfde signaal als hierboven

# # Bereken wavelet decomposities
# coif8_coeffs = pywt.wavedec(signal, wavelet='coif8', level=3)
# db14_coeffs = pywt.wavedec(signal, wavelet='db14', level=3)

# # Reconstrueer approximaties
# coif8_reconstructed = pywt.waverec([coif8_coeffs[0]] + [None] * 3, 'coif8')
# db14_reconstructed = pywt.waverec([db14_coeffs[0]] + [None] * 3, 'db14')

# # Plot beide reconstructies over elkaar
# plt.plot(coif8_reconstructed[:len(signal)], label='coif8', alpha=0.7)
# plt.plot(db14_reconstructed[:len(signal)], label='db14', alpha=0.7)
# plt.title("Vergelijking van coif8 en db14 wavelet reconstructies (level 3)")
# plt.legend()
# plt.show()

# # Voeg wat nuttige analyses toe
# print("\nSamenvatting van de dataset:")
# print(f"Totaal aantal files: {len(df_measurement_files)}")
# print(f"\nGemiddelde duur per meting: {df_measurement_files['duration_sec'].mean():.2f} seconden")
# print(f"Standaard deviatie van duur: {df_measurement_files['duration_sec'].std():.2f} seconden")
# print(f"\nAantal kanalen per meting: {df_measurement_files['channels'].value_counts().to_dict()}")

# # Visualisatie van de duur van metingen
# plt.figure(figsize=(10, 5))
# plt.hist(df_measurement_files['duration_sec'], bins=50)
# plt.title('Distributie van meting duraties')
# plt.xlabel('Duratie (seconden)')
# plt.ylabel('Aantal files')
# plt.show()

# # Boxplot van duur per operatie
# plt.figure(figsize=(15, 6))
# sns.boxplot(data=df_measurement_files, x='operation', y='duration_sec')
# plt.xticks(rotation=45)
# plt.title('Duratie per operatie')
# plt.xlabel('Operatie')
# plt.ylabel('Duratie (seconden)')
# plt.show()

# # Maak een bar plot van de verdeling van de data over machines, operaties en klassen
# import matplotlib.pyplot as plt

# # Maak een figuur met subplots
# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# # Plot voor machine verdeling
# df_measurement_files['machine'].value_counts().plot(kind='bar', ax=ax1)
# ax1.set_title('Aantal samples per machine')
# ax1.set_xlabel('Machine')
# ax1.set_ylabel('Aantal samples')
# ax1.tick_params(axis='x', rotation=45)

# # Plot voor operatie verdeling 
# df_measurement_files['operation'].value_counts().plot(kind='bar', ax=ax2)
# ax2.set_title('Aantal samples per operatie')
# ax2.set_xlabel('Operatie')
# ax2.set_ylabel('Aantal samples')
# ax2.tick_params(axis='x', rotation=45)

# # Plot voor klasse verdeling
# df_measurement_files['class'].value_counts().plot(kind='bar', ax=ax3)
# ax3.set_title('Aantal samples per klasse')
# ax3.set_xlabel('Klasse')
# ax3.set_ylabel('Aantal samples')
# ax3.tick_params(axis='x', rotation=45)

# # Plot voor operatie verdeling 
# df_measurement_files['operation'].value_counts().plot(kind='bar', ax=ax2)
# ax2.set_title('Aantal samples per operatie')
# ax2.set_xlabel('Operatie')
# ax2.set_ylabel('Aantal samples')

# # Plot voor klasse verdeling
# df_measurement_files['class'].value_counts().plot(kind='bar', ax=ax3)
# ax3.set_title('Aantal samples per klasse')
# ax3.set_xlabel('Klasse')
# ax3.set_ylabel('Aantal samples')

# fig.tight_layout()
# plt.show()

# # Export the plot to a file
# # fig.savefig('data_distribution.png')

# # Print wat statistieken
# print("\nDataset statistieken:")
# print(f"Totaal aantal samples: {len(df_measurement_files)}")
# print("\nVerdeling per machine:")
# print(df_measurement_files['machine'].value_counts())
# print("\nVerdeling per operatie:")
# print(df_measurement_files['operation'].value_counts())
# print("\nVerdeling per klasse:")
# print(df_measurement_files['class'].value_counts())


# print("mean:",df_measurement_files['measurements'].mean())
# print("std:",df_measurement_files['measurements'].std())
# print("median:",df_measurement_files['measurements'].median())